# Alice Gets a Transcript

A credential is a piece of information about an identity — a name, an age, a credit score… It is information claimed to be true. In this case, the credential is named, "Transcript".

Credentials are offered by an issuer.

An issuer may be any identity owner known to the Ledger and any issuer may issue a credential about any identity owner it can identify.

The usefulness and reliability of a credential are tied to the reputation of the issuer with respect to the credential at hand.
For Alice to self-issue a credential that she likes chocolate ice cream may be perfectly reasonable, but for her to self-issue a credential that she graduated from Faber College should not impress anyone.


As we mentioned in [About Alice](#about-alice) **Alice** graduated from **Faber College**.
After **Faber College** had established a connection with Alice, it created for her a Credential Offer about the issuance of the **Transcript** Credential.
```python
  # Faber Agent
  transcript_cred_offer_json = await anoncreds.issuer_create_credential_offer(faber_wallet, faber_transcript_cred_def_id)
```

**Note:** All messages sent between actors are encrypted using `Authenticated-encryption` scheme.

The value of this **Transcript** Credential is that it is provably issued by **Faber College**.

**Alice** wants to see the attributes that the **Transcript** Credential contains.
These attributes are known because a Credential Schema for **Transcript** has been written to the Ledger.
```python
  # Alice Agent
  get_schema_request = await ledger.build_get_schema_request(alice_faber_did, transcript_cred_offer['schema_id'])
  get_schema_response = await ledger.submit_request(pool_handle, get_schema_request)
  transcript_schema = await ledger.parse_get_schema_response(get_schema_response)

  print(transcript_schema['data'])
  # Transcript Schema:
  {
      'name': 'Transcript',
      'version': '1.2',
      'attr_names': ['first_name', 'last_name', 'degree', 'status', 'year', 'average', 'ssn']
  }
```

However, the **Transcript** Credential has not been delivered to Alice yet in a usable form. Alice wants to use that Credential.
To get it, Alice needs to request it, but first she must create a **Master Secret**.

**Note:** A Master Secret is an item of Private Data used by a Prover to guarantee that a credential uniquely applies to them. The Master Secret is an input that combines data from multiple Credentials in order to prove that the Credentials have a common subject (the Prover). A Master Secret should be known only to the Prover.

Alice creates Master Secret in her wallet.
```python
  # Alice Agent
  alice_master_secret_id = await anoncreds.prover_create_master_secret(alice_wallet, None)
```

Alice also needs to get the Credential Definition corresponding to the `cred_def_id` in the **Transcript** Credential Offer.
```python
  # Alice Agent
  get_cred_def_request = await ledger.build_get_cred_def_request(alice_faber_did, transcript_cred_offer['cred_def_id'])
  get_cred_def_response = await ledger.submit_request(pool_handle, get_cred_def_request)
  faber_transcript_cred_def = await ledger.parse_get_cred_def_response(get_cred_def_response)
```

Now Alice has everything to create a Credential Request of the issuance of the **Faber Transcript** Credential.
```python   
  # Alice Agent
    (transcript_cred_request_json, transcript_cred_request_metadata_json) = \
        await anoncreds.prover_create_credential_req(alice_wallet, alice_faber_did, transcript_cred_offer_json,
                                                     faber_transcript_cred_def, alice_master_secret_id)
```

**Faber** prepares both raw and encoded values for each attribute in the **Transcript** Credential Schema.
**Faber** creates the **Transcript** Credential for Alice.
```python
  # Faber Agent
  transcript_cred_values = json.dumps({
      "first_name": {"raw": "Alice", "encoded": "1139481716457488690172217916278103335"},
      "last_name": {"raw": "Garcia", "encoded": "5321642780241790123587902456789123452"},
      "degree": {"raw": "Bachelor of Science, Marketing", "encoded": "12434523576212321"},
      "status": {"raw": "graduated", "encoded": "2213454313412354"},
      "ssn": {"raw": "123-45-6789", "encoded": "3124141231422543541"},
      "year": {"raw": "2015", "encoded": "2015"},
      "average": {"raw": "5", "encoded": "5"}
  })

  transcript_cred_json, _, _ = \
      await anoncreds.issuer_create_credential(faber_wallet, transcript_cred_offer_json, transcript_cred_request_json,
                                               transcript_cred_values, None, None)
```

Now the **Transcript** Credential has been issued. Alice stores it in her wallet.
```python
  # Alice Agent
  await anoncreds.prover_store_credential(alice_wallet, None, transcript_cred_request_json, transcript_cred_request_metadata_json,
                                          transcript_cred_json, faber_transcript_cred_def, None)
```

Alice has it in her possession, in much the same way that she would hold a physical transcript that had been mailed to her.

[previous](07-cred-definition-setup.ipynb) | [next](09-alice-applies-for-a-job.ipynb)